# **1. Prepare the environment**

## **1.1 Fetch the "Label-Bot" repo and modify it so that it can be used in colab**

In [ ]:
!git clone https://github.com/GiorgosKarantonis/Label-Bot

!mv Label-Bot Label_Bot
!touch Label_Bot/__init__.py

## **1.2 Mount google drive in order to be able to access the preprocessed dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **2. Prepare the Label Bot**

## **2.1 Import all the required libraries**

In [ ]:
import pandas as pd

try:
    from transformers import BertTokenizer, TFBertForSequenceClassification
except:
    !pip install transformers==3.0.0
    from transformers import BertTokenizer, TFBertForSequenceClassification

import Label_Bot.preprocessing as pp

In [14]:
MEMORY_LIMIT = 100

# a dummy set of labels used for the proof of concept
LABELS = [
    'bug', 
    'enhancement', 
    'question'
]

In [15]:
df = pp.load_data(memory_limit=MEMORY_LIMIT, file='./drive/My Drive/Label Bot/data/github.pkl')

for c in df.columns:
    if c.startswith('label_'):
        df = df.drop(c, axis=1)

df.head()

url  ... issue_number
0  https://github.com/F5Networks/f5-openstack-lba...  ...          835
1          https://github.com/aspnet/Mvc/issues/6339  ...         6339
2  https://github.com/ionic-team/ionic-cli/issues...  ...         3044
3     https://github.com/thefarwind/chip-8/issues/21  ...           21
4   https://github.com/n-sokolov/CoffeeShop/issues/1  ...            1

[5 rows x 8 columns]

In [16]:
df = pp.clean_text_data(df, 'title', 'body')

df.head()

url  ... issue_number
0  https://github.com/F5Networks/f5-openstack-lba...  ...          835
1          https://github.com/aspnet/Mvc/issues/6339  ...         6339
2  https://github.com/ionic-team/ionic-cli/issues...  ...         3044
3     https://github.com/thefarwind/chip-8/issues/21  ...           21
4   https://github.com/n-sokolov/CoffeeShop/issues/1  ...            1

[5 rows x 8 columns]

In [17]:
unique_labels = pp.get_unique_values(df, 'labels').keys().values

pd.Series(unique_labels)

0                               bug
1                       enhancement
2                          question
3                          kind/bug
4                             fixed
                  ...              
80                             done
81                       discussion
82                      kind/bug/p0
83                      sig/network
84    role-based-access-control/svc
Length: 85, dtype: object

In [ ]:
paraphrase_list = pp.make_combinations(LABELS, unique_labels)
_, paraphrase_likelihood = pp.check_paraphrase(paraphrase_list)

In [19]:
label_mapping = {}
for i, pair in enumerate(paraphrase_list):
    if paraphrase_likelihood[i] > .5:
        target_l, real_l = pair[0], pair[1]
        try:
            label_mapping[real_l].append((target_l, paraphrase_likelihood[i]))
        except:
            label_mapping[real_l] = []
            label_mapping[real_l].append((target_l, paraphrase_likelihood[i]))

pd.DataFrame(label_mapping).T

0
bug                        (bug, 0.9162983)
kind/bug                  (bug, 0.84865934)
extension-bug             (bug, 0.62284034)
type:visual bug            (bug, 0.7715802)
minor bug                  (bug, 0.5306739)
bug report                 (bug, 0.6632633)
type: bug                 (bug, 0.82417744)
type-bug                    (bug, 0.799179)
enhancement        (enhancement, 0.9370464)
kind/enhancement  (enhancement, 0.90538484)
doc-enhancement    (enhancement, 0.8313373)
question              (question, 0.9274783)

In [21]:
df['labels']

0                 [s3, p3, test-bug]
1                         [question]
2                              [bug]
3                       [bug, tests]
4                      [enhancement]
                   ...              
95    [extension-bug, status: stale]
96                       [bug, done]
97                     [enhancement]
98              [bug, pdf previewer]
99                             [bug]
Name: labels, Length: 100, dtype: object

In [22]:
label_mapping = pp.disambiguate_labels(label_mapping)
df['labels'] = pp.map_labels(df['labels'], label_mapping)

df['labels']

0       [undefined]
1        [question]
2             [bug]
3             [bug]
4     [enhancement]
          ...      
95            [bug]
96            [bug]
97    [enhancement]
98            [bug]
99            [bug]
Name: labels, Length: 100, dtype: object

In [23]:
if 'undefined' not in LABELS:
    LABELS.append('undefined')

labels_vectorized = pp.vectorize(df['labels'], LABELS, prefix='label')
df = pp.transform(df, to_add=[labels_vectorized])

df.head()

Vectorizing Features...

Transforming DataFrame...



url  ... label_undefined
0  https://github.com/F5Networks/f5-openstack-lba...  ...             1.0
1          https://github.com/aspnet/Mvc/issues/6339  ...             0.0
2  https://github.com/ionic-team/ionic-cli/issues...  ...             0.0
3     https://github.com/thefarwind/chip-8/issues/21  ...             0.0
4   https://github.com/n-sokolov/CoffeeShop/issues/1  ...             0.0

[5 rows x 12 columns]